In [93]:
#defining the classes Package, Bin and Student

class Package:
    def __init__(self, id, bin, location_in_bin, size, student):
        self.id = id
        self.size = size
        self.student = student  #student object
        self.letter = student.lastname_letter
        self.bin = bin
        self.location_in_bin = location_in_bin
        
    def getID(self):
        return self.id

    def getSize(self):
        return self.size
    
    def getLetter(self):
        return self.letter
    
    def getBin(self):
        return self.bin

    def getStudent(self):
        return self.student

    # going to be used to modify Package when something is removed from its bin
    def setLocation(self, value):
        self.location_in_bin = value
    
    def __str__(self):
        return f"packageID: {self.id} | size = {self.size} | bin = {self.bin}, {self.location_in_bin}"

    
class Bin:
    packages = []
    def __init__(self,label):
        self.label = label
    
    def addPackage(self,package):
        self.packages.append(package)
    
    def removePackage(self,package):
        self.packages.remove(package)
    
    def getPackages(self):
        return self.packages

    def getNumPackages(self):
        return len(self.packages)

    def __str__(self):
        return f"binID: {self.label}"

class Student: 
    def __init__(self,id,lastname_letter):
        self.id = id
        self.lastname_letter = lastname_letter.upper()
    def __str__(self):
        return f"studentID: {self.id} | {self.lastname_letter}"

In [94]:
# create student
test_student = Student(0, "L")
print(test_student)

# create bins
bins = []
for i in range(10):
    bins.append(Bin(i))

bins[0].getNumPackages()

#create package with id=0, put it in bin j=0
j = 0
test_package = Package(0, j, bins[j].getNumPackages(), 'small', test_student)
print(test_package)

studentID: 0 | L
packageID: 0 | size = small | bin = 0, 0


In [95]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_csv('lastnames.csv')
df['Percentage'] = df['Percentage']*0.01
df.head()

,Letter,People,Percentage
0,M,"25,610,240",0.096
1,S,"25,056,728",0.094
2,B,"22,587,641",0.085
3,C,"20,582,563",0.077
4,H,"18,810,320",0.071


In [97]:
def runLastNameSimulation(packagesize_dist, lastname_dist, num_bins, num_packages):
    # for now, packagesize_dist is not implemented.
    # lastname will be implemented (but isn't rn) 
    
    all_bins = [] #to store all the bins of this simulation
    #splits the alphabet into bins
    alpha_bin_names = np.array_split(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), num_bins)
    #creates bins based on alphabet
    for name in alpha_bin_names:
       all_bins.append(Bin(str(name)))
    
    all_packages = [] #to store all the packages of this simulation
    all_students = [] #to store all the students of this simulation
    for i in range(num_packages):
        #randomly get the lastnameletter from the distribution 
        # WORK IN PROGRESS, FOR NOW, CONSTANT --------------------------------------------------------------------------COME BACK TO
        letter = 'M'

        #make the student 
        all_students.append(Student(i, letter))

        #determine which bin it goes in
        for j in range(num_bins):
            if letter in alpha_bin_names[j]:
                bin_to_go_in = j #this returns the bin's index number in the all_bins list 

        #finally, make the package object
        package = (Package(i, bin_to_go_in, all_bins[bin_to_go_in].getNumPackages(), 'small', all_students[i]))
        all_packages.append(package)
        
        #update the bin so it knows it has a new package
        all_bins[bin_to_go_in].addPackage(package)

#print where the packages were assigned to!
    for p in all_packages:
        print(p)



In [98]:
runLastNameSimulation('test', 'test', 10, 5)

packageID: 0 | size = small | bin = 4, 0
packageID: 1 | size = small | bin = 4, 1
packageID: 2 | size = small | bin = 4, 2
packageID: 3 | size = small | bin = 4, 3
packageID: 4 | size = small | bin = 4, 4
